# Thanaweya Amma Grades Analysis

In [6]:
# Import libraries
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time
import matplotlib.pyplot as plt
import random
random.seed(42)

## Data Scraping

In [7]:
with open('extracted.txt', 'r') as f:
    names = [line.strip() for line in f]

In [8]:
# Initiate the bot
url = 'https://gtdb.ecogenomic.org/advanced?exp=KDEmMiYzJig2KSk~&1=NX42fm51bGw~&2=N345fjU~&3=Nn4xMX45NQ~~&6=Mn4yfmtraw~~'
driver = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver')
driver.get(url)
action = webdriver.ActionChains(driver)

/var/folders/f0/9mpvthp932lfvdkms8zjb2600000gn/T/ipykernel_62204/3412027653.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver')


In [20]:
inpt = driver.find_element(By.XPATH, '/html/body/div/div/div/div/main/div/div/div/div[1]/div/div/div[2]/div/div/div[5]/div/div[2]/div/div/div[3]/div/div/div/div[1]/input')
submit = driver.find_element(By.XPATH, '//*[@id="app"]/div/main/div/div/div/div[1]/div/div/div[3]/button/span')
inpt.click()
inpt.clear()
inpt.send_keys(Keys.COMMAND, 'a')
inpt.send_keys(Keys.DELETE)
inpt.send_keys(names[0])
submit.click()
time.sleep(4)
download = driver.find_element(By.XPATH, '//*[@id="app"]/div[1]/main/div/div/div/div[2]/div/div[1]/div/button/span')
download.click()
time.sleep(3)
##curl = driver.find_element(By.XPATH, '//*[@id="app"]/div[3]/div/div/div[3]/button[1]')
#nowhere = driver.find_element(By.XPATH, '//*[@id="app"]/div[4]/div/div/div[2]/div/div/div[5]/div/div[1]/label')
action.key_down(Keys.COMMAND).click(curl).key_up(Keys.COMMAND).perform()
time.sleep(1.5)
nowhere.click()
nowhere.click()
webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
print(f'Downloaded {names[0]}')

for name in names[1:]:
    try:
        webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        inpt.click()
        inpt.clear()
        inpt.send_keys(Keys.COMMAND, 'a')
        inpt.send_keys(Keys.DELETE)
        inpt.send_keys(name)
        submit.click()
        time.sleep(9)
        download.click()
        time.sleep(0.8)
        action.key_down(Keys.COMMAND).click(curl).key_up(Keys.COMMAND).perform()
        time.sleep(0.1)
        nowhere.click() 
        nowhere.click()
        webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        print(f'Downloaded: {name}')
    except:
        print(f'Failed: {name}')

Downloaded s__Bacillus_anthracis
Downloaded: s__Bacillus_cereus
Downloaded: s__Bartonella_henselae
Downloaded: s__Bartonella_quintana
Downloaded: s__Bordetella_pertussis
Failed: s__Borrelia_burgdorferi
Failed: s__Borrelia_garinii
Failed: s__Borrelia_afzelii
Downloaded: s__Borrelia_recurrentis
Failed: s__Brucella_abortus
Failed: s__Brucella_canis
Downloaded: s__Brucella_melitensis
Failed: s__Brucella_suis
Downloaded: s__Campylobacter_jejuni
Downloaded: s__Chlamydia_pneumoniae
Downloaded: s__Chlamydia_trachomatis
Failed: s__Chlamydophila_psittaci
Downloaded: s__Clostridium_botulinum
Failed: s__Clostridium_difficile
Downloaded: s__Clostridium_perfringens
Downloaded: s__Clostridium_tetani
Downloaded: s__Corynebacterium_diphtheriae
Downloaded: s__Enterococcus_faecalis
Downloaded: s__Enterococcus_faecium
Downloaded: s__Escherichia_coli
Downloaded: s__Francisella_tularensis
Downloaded: s__Haemophilus_influenzae
Downloaded: s__Helicobacter_pylori
Downloaded: s__Legionella_pneumophila
Downloade

In [18]:
nowhere = driver.find_element(By.XPATH, '//*[@id="app"]/div[3]/div/div/div[2]/div/div/div[5]/div/div[1]/label')


In [ ]:
with open('out.txt', 'r') as f:
    out